In [76]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [77]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
%run drive/MyDrive/ERSP/violin-renderer/src/parse.ipynb

In [78]:
# alright, let's try with one movement, one performer!
X = torch.Tensor(parseInputFile('drive/MyDrive/ERSP/violin-renderer/bach-violin-dataset/dataset/source-input/emil-telmanyi/bwv1002/emil-telmanyi_bwv1002_mov1.csv'))
y = torch.Tensor(parseGroundtruthFile('drive/MyDrive/ERSP/violin-renderer/bach-violin-dataset/dataset/ground-truth/emil-telmanyi/bwv1002/emil-telmanyi_bwv1002_mov1.csv'))

In [79]:
print(X)
print(y)
print(X.shape)
print(y.shape)

tensor([[   0.,    6.,   78.],
        [   6.,   24.,   59.],
        [   6.,   24.,   66.],
        ...,
        [2262., 2298.,   66.],
        [2262., 2298.,   74.],
        [2262., 2298.,   83.]])
tensor([[1.9200e-01, 9.2800e-01],
        [9.6000e-01, 2.4640e+00],
        [9.6000e-01, 2.4640e+00],
        ...,
        [1.9200e+02, 1.9498e+02],
        [1.9200e+02, 1.9498e+02],
        [1.9200e+02, 1.9498e+02]])
torch.Size([502, 3])
torch.Size([502, 2])


In [80]:
# Starting with linear regression. For this, we only need one linear layer.
# For now, we don't need to build our own module yet because linear regression is simple. However, for an MLP, we will need multiple layers.
model = nn.Linear(3, 2)

In [81]:
# it was retaining nan values for its parameters...
model.reset_parameters()

In [82]:
# Define our loss function (mean squared error) to be used in the grad descent step
loss = nn.MSELoss()

# Performs the gradient descent steps
optimizer = torch.optim.SGD(model.parameters(), lr=1e-7)

In [83]:
# Template code from https://uvadlc-notebooks.readthedocs.io/en/latest/tutorial_notebooks/tutorial2/Introduction_to_PyTorch.html#The-Basics-of-PyTorch
def train_model(model, optimizer, input_notes, truth, loss_module, num_epochs=50000):
    # Set model to train mode
    model.train()

    # Training loop
    for epoch in range(num_epochs):

        ## Step 1: Move input data to device (only strictly necessary if we use GPU)
        # data_inputs = data_inputs.to(device)
        # data_labels = data_labels.to(device)

        ## Step 2: Run the model on the input data
        preds = model(input_notes)

        ## Step 3: Calculate the loss
        loss = loss_module(preds, y)
        print(loss)

        ## Step 4: Perform backpropagation
        # Before calculating the gradients, we need to ensure that they are all zero.
        # The gradients would not be overwritten, but actually added to the existing ones.
        optimizer.zero_grad()
        # Perform backpropagation
        loss.backward()

        ## Step 5: Update the parameters
        optimizer.step()

In [84]:
train_model(model, optimizer, X, y, loss)

Streaming output truncated to the last 5000 lines.
tensor(2.2792, grad_fn=<MseLossBackward0>)
tensor(2.2792, grad_fn=<MseLossBackward0>)
tensor(2.2791, grad_fn=<MseLossBackward0>)
tensor(2.2791, grad_fn=<MseLossBackward0>)
tensor(2.2791, grad_fn=<MseLossBackward0>)
tensor(2.2791, grad_fn=<MseLossBackward0>)
tensor(2.2791, grad_fn=<MseLossBackward0>)
tensor(2.2791, grad_fn=<MseLossBackward0>)
tensor(2.2791, grad_fn=<MseLossBackward0>)
tensor(2.2791, grad_fn=<MseLossBackward0>)
tensor(2.2791, grad_fn=<MseLossBackward0>)
tensor(2.2791, grad_fn=<MseLossBackward0>)
tensor(2.2791, grad_fn=<MseLossBackward0>)
tensor(2.2791, grad_fn=<MseLossBackward0>)
tensor(2.2791, grad_fn=<MseLossBackward0>)
tensor(2.2791, grad_fn=<MseLossBackward0>)
tensor(2.2791, grad_fn=<MseLossBackward0>)
tensor(2.2791, grad_fn=<MseLossBackward0>)
tensor(2.2791, grad_fn=<MseLossBackward0>)
tensor(2.2791, grad_fn=<MseLossBackward0>)
tensor(2.2791, grad_fn=<MseLossBackward0>)
tensor(2.2791, grad_fn=<MseLossBackward0>)
ten

In [85]:
X_test = torch.Tensor(parseInputFile('drive/MyDrive/ERSP/violin-renderer/bach-violin-dataset/dataset/source-input/emil-telmanyi/bwv1001/emil-telmanyi_bwv1001_mov3.csv'))
y_test = torch.Tensor.tolist(model(X_test))

print(model.weight)

X_test = torch.Tensor.tolist(X_test)
print(X_test)
for i in range(len(y_test)):
    y_test[i].append(X_test[i][2])

print(y_test)

Parameter containing:
tensor([[-0.2097,  0.2936, -0.0389],
        [ 0.0877, -0.0039, -0.0060]], requires_grad=True)
[[0.0, 18.0, 58.0], [18.0, 24.0, 62.0], [24.0, 36.0, 65.0], [36.0, 42.0, 63.0], [42.0, 48.0, 62.0], [48.0, 54.0, 60.0], [54.0, 60.0, 58.0], [60.0, 72.0, 74.0], [60.0, 72.0, 82.0], [72.0, 84.0, 65.0], [72.0, 84.0, 74.0], [72.0, 84.0, 82.0], [84.0, 96.0, 72.0], [84.0, 96.0, 81.0], [96.0, 108.0, 70.0], [96.0, 108.0, 79.0], [108.0, 120.0, 70.0], [108.0, 120.0, 79.0], [120.0, 132.0, 69.0], [120.0, 132.0, 77.0], [132.0, 144.0, 63.0], [144.0, 156.0, 62.0], [156.0, 168.0, 72.0], [156.0, 168.0, 77.0], [168.0, 180.0, 70.0], [168.0, 180.0, 79.0], [180.0, 192.0, 60.0], [192.0, 198.0, 75.0], [192.0, 204.0, 69.0], [198.0, 204.0, 72.0], [204.0, 216.0, 58.0], [204.0, 216.0, 70.0], [204.0, 216.0, 74.0], [216.0, 228.0, 65.0], [228.0, 234.0, 74.0], [228.0, 246.0, 70.0], [234.0, 240.0, 77.0], [240.0, 246.0, 75.0], [246.0, 252.0, 70.0], [246.0, 252.0, 74.0], [252.0, 264.0, 69.0], [252.0, 264